In [1]:
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import vbfprocessor
import awkward as ak
import uproot

import os
import time

In [2]:
all_filesets = {
    "VBF_SMEFTsim_topU3l_NP1": [
        "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/VBF_SMEFTsim_topU3l_NP1/VBF_SMEFTsim_topU3l_NP1/230526_132704/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHbox": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Direct_cHbox/VBF_SMEFTsim_topU3l_Direct_cHbox/230919_233803/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Linear_cHW": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHW/VBF_SMEFTsim_topU3l_Linear_cHW/230920_001033/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Quad_cHW": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHW/VBF_SMEFTsim_topU3l_Quad_cHW/230920_001106/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Linear_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHWtil/VBF_SMEFTsim_topU3l_Linear_cHWtil/230920_003136/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Quad_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHWtil/VBF_SMEFTsim_topU3l_Quad_cHWtil/230920_003225/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],   
    "VBF_SMEFTsim_topU3l_Linear_cHbox": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHbox/VBF_SMEFTsim_topU3l_Linear_cHbox/230919_235238/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Quad_cHbox": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHbox/VBF_SMEFTsim_topU3l_Quad_cHbox/230919_235331/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],  
    "VBF_SMEFTsim_topU3l_Linear_cHj1": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHj1/VBF_SMEFTsim_topU3l_Linear_cHj1/230919_233248/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ], 
    "VBF_SMEFTsim_topU3l_Quad_cHj1": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHj1/VBF_SMEFTsim_topU3l_Quad_cHj1/230919_233036/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Linear_cHj3": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Linear_cHj3/VBF_SMEFTsim_topU3l_Linear_cHj3/230919_233413/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ], 
    "VBF_SMEFTsim_topU3l_Quad_cHj3": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBF_SMEFTsim_topU3l_Quad_cHj3/VBF_SMEFTsim_topU3l_Quad_cHj3/230919_233144/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHj1": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBFH_SMEFTsim_NPall_topU3l_cHj1_1/VBFH_SMEFTsim_NPall_topU3l_cHj1_1/230519_134829/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHj3": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBFH_SMEFTsim_NPall_topU3l_cHj3_1/VBFH_SMEFTsim_NPall_topU3l_cHj3_1/230519_134845/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHW": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBFH_SMEFTsim_NPall_topU3l_cHW_1/VBFH_SMEFTsim_NPall_topU3l_cHW_1/230519_134915/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "VBF_SMEFTsim_topU3l_Direct_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jennetd/vbf-eft/VBFH_SMEFTsim_NPall_topU3l_cHWtil_1/VBFH_SMEFTsim_NPall_topU3l_cHWtil_1/230519_134931/0000//VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_NP1": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_NP1/Hqq_SMEFTsim_topU3l_NP1/230918_201208/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHW": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHW/Hqq_SMEFTsim_topU3l_Direct_cHW/230918_220845/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHWtil": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHWtil/Hqq_SMEFTsim_topU3l_Direct_cHWtil/230918_220906/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHbox": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHbox/Hqq_SMEFTsim_topU3l_Direct_cHbox/230918_220925/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHj1": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHj1/Hqq_SMEFTsim_topU3l_Direct_cHj1/230918_221045/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
    "Hqq_SMEFTsim_topU3l_Direct_cHj3": [
         "root://cmseos.fnal.gov//store/user/jdickins/vbf-eft/Hqq_SMEFTsim_topU3l_Direct_cHj3/Hqq_SMEFTsim_topU3l_Direct_cHj3/230918_221110/0000/VBFH_SMEFTsim_"+str(i)+".root" for i in range(1,201)
    ],
}

In [3]:
def run_samp(sample):

    fileset = {}
    fileset[sample] = all_filesets[sample]
    outfile = "coffea/"+sample+".coffea"
    
    p = vbfprocessor.VBFProcessor(isMC=True)
    args = {'savemetrics':True, 'schema':NanoAODSchema}

    output = processor.run_uproot_job(
        fileset,
        treename="Events",
        processor_instance=p,
        executor=processor.iterative_executor,#executor=processor.dask_executor,
        executor_args={'schema': NanoAODSchema,'workers': 4},
        chunksize=50000,
        )

    util.save(output, outfile)
    print("saved " + outfile)

In [ ]:
for s in all_filesets.keys():
    
    outfile = "coffea/"+s+".coffea"
    if not os.path.exists(outfile) and 'Hqq' in s:
        print(s)
        run_samp(s)

Processing  36% ━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 72/200 [ 0:16:14 < 0:29:05 | 0.1 chunk/s ]

/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-uproot/4.2.2-2c5761a9de164cd2dd3fc5b9fb4a4ff8/lib/python3.9/site-packages/uproot/reading.py:171: FutureWarning: XRootD unknown is not fully supported; either upgrade to 5.2.0+ or set

    open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource

  warnings.warn(message, FutureWarning)
